In [1]:
import sys
sys.path.insert(0, '../../')
import ccal
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

import numpy as np
import pandas as pd

from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, fcluster, cophenet
from scipy.spatial.distance import pdist

==================== Computational Cancer Analysis Library ====================
<14:34:13> Checking dependencies ...
<14:34:13> Using the following packages:
<14:34:13> 	matplotlib (v1.5.1)
<14:34:13> 	numpy (v1.10.4)
<14:34:13> 	pandas (v0.18.0)
<14:34:13> 	rpy2 (v2.7.9)
<14:34:13> 	scikit-learn (v0.17.1)
<14:34:13> 	scipy (v0.17.0)
<14:34:13> 	seaborn (v0.7.0)


In [2]:
kmin= 1
kmax = 5
kincrement = 1
nclustering = 3
h_matrix = ccal.support.read_gct('../../../hematopoietic_cancer/result/ccle_hema/k9/ccle_hema_k_9_H.gct')

In [3]:
results = pd.DataFrame(index=range(kmin, kmax + 1, kincrement), columns=list(h_matrix.index) + ['cophenetic_correlation'])

standardized_h_matrix = ccal.support.standardize_pandas_object(h_matrix)
standardized_and_clipped_h_matrix = standardized_h_matrix.clip(-3, 3)
compared_matrix = ccal.analyze.compare_matrices(standardized_and_clipped_h_matrix, standardized_and_clipped_h_matrix)

for k in range(kmin, kmax + 1, kincrement):
    print(k)
    labels = pd.DataFrame(index=range(nclustering), columns=compared_matrix.index)
    for i in range(nclustering):
        ward = AgglomerativeClustering(n_clusters=k)
        ward.fit(compared_matrix)
        labels.iloc[i, :] = ward.labels_

    cooccurences = pd.DataFrame(index=compared_matrix.index, columns=compared_matrix.columns)
    cooccurences.fillna(0, inplace=True)
    for n, s in labels.iterrows():
        for i in s.index:
            for j in s.index:
                if i is j or s.ix[i] == s.ix[j]:
                    cooccurences.ix[i, j] += 1
    cooccurences /= nclustering

    distances = 1 - cooccurences
    link = linkage(distances, method='ward')
    labels = fcluster(link, k, criterion='maxclust')
    cophenetic_correlation = cophenet(link, pdist(distances))[0]
    results.ix[k, h_matrix.index] = labels
    results.ix[k, 'cophenetic_correlation'] = cophenetic_correlation

results

<14:34:14> Features 1 nmf_compC1_9 (1/9) vs. features 2 ...
<14:34:14> Features 1 nmf_compC2_9 (2/9) vs. features 2 ...
<14:34:14> Features 1 nmf_compC3_9 (3/9) vs. features 2 ...
<14:34:14> Features 1 nmf_compC4_9 (4/9) vs. features 2 ...
<14:34:14> Features 1 nmf_compC5_9 (5/9) vs. features 2 ...
<14:34:14> Features 1 nmf_compC6_9 (6/9) vs. features 2 ...
<14:34:14> Features 1 nmf_compC7_9 (7/9) vs. features 2 ...
<14:34:14> Features 1 nmf_compC8_9 (8/9) vs. features 2 ...
<14:34:14> Features 1 nmf_compC9_9 (9/9) vs. features 2 ...
1
2
3
4
5


,nmf_compC1_9,nmf_compC2_9,nmf_compC3_9,nmf_compC4_9,nmf_compC5_9,nmf_compC6_9,nmf_compC7_9,nmf_compC8_9,nmf_compC9_9,cophenetic_correlation
1,1,1,1,1,1,1,1,1,1,NaN
2,1,1,2,2,2,1,2,2,1,1
3,2,3,1,1,1,3,1,1,2,0.989784
4,2,3,1,1,4,3,4,1,2,0.978958
5,2,3,1,1,4,3,5,1,2,0.95529
